In [1]:
import pandas as pd
from bs4 import BeautifulSoup

import asyncio
import nest_asyncio
import aiohttp

In [2]:
history = {
    'authors': [],
    'titles': [],
    'annotations': [],
    'image_urls': [],
    'book_urls': [],
    'genres': []
}

In [3]:
# сюда транслит название с сайты
genres = {
    'Боевое фэнтези': 'boevoe-fentezi',
    'Историческое фэнтези': 'istoricheskoe-fentezi'
}

MAIN_GENRE = 'Фэнтези'
SUB_GENRE = "Историческое фэнтези "
NAME = genres['Историческое фэнтези'] # тут менять на название с сайта
PAGES = 90 # выставить кол-во стр, которые нужно спарсить

In [4]:
def save_book_data(history, books):
    BASE_URL = 'https://litnet.com'
    
    for book in books:
        author = book.find_all('a', class_='author')
        author = ', '.join([a.get_text() for a in author])
        
        title = book.find('h4', class_='book-title').get_text(strip=True)
        annotation = book.find('span', itemprop='description').get_text(strip=True)
        image_url = book.find('img', itemprop='image')['src']
        book_url = BASE_URL + books[0].find('a', class_='readmore')['href']
        
        history['authors'].append(author)
        history['titles'].append(title)
        history['annotations'].append(annotation)
        history['image_urls'].append(image_url)
        history['book_urls'].append(book_url)
        history['genres'].append(f'{MAIN_GENRE}, {SUB_GENRE}')

In [5]:
semaphore = asyncio.Semaphore(10)

async def fetch(session, url):
    async with semaphore:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            books = soup.find_all('div', class_='row book-item')
            save_book_data(history, books)
        
        await asyncio.sleep(0.5)

In [6]:
async def main(url, genre, n_pages=10):
    urls = [url.format(genre, genre, i) for i in range(1, n_pages)]
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        await asyncio.gather(*tasks)

In [7]:
url = 'https://litnet.com/ru/top/{}?alias={}&page={}'

nest_asyncio.apply()
await main(url, NAME, PAGES)

In [8]:
books_df = pd.DataFrame({
    'author': history['authors'],
    'title': history['titles'],
    'annotation': history['annotations'],
    'img_url': history['image_urls'],
    'book_url': history['book_urls'],
    'genre': history['genres']
})

In [9]:
books_df

,author,title,annotation,img_url,book_url,genre
0,Арина Теплова,Наследница Дивной усадьбы,Я очутилась в теле юной графини Шереметьевой. ...,https://rust.litnet.com/uploads/covers/120/171...,https://litnet.com/ru/book/naslednica-divnoi-u...,"Фэнтези, Историческое фэнтези"
1,Яна Павлова,Все люди как люди. Княгиня,Высшие силы дали Акулине шанс - если она справ...,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/naslednica-divnoi-u...,"Фэнтези, Историческое фэнтези"
2,Боярова Мелина,Брак по расчету. Счастью не прикажешь,Рядовая поездка в приют для одаренных сирот пр...,https://rust.litnet.com/uploads/covers/120/171...,https://litnet.com/ru/book/naslednica-divnoi-u...,"Фэнтези, Историческое фэнтези"
3,Теплая Елена,Когда-то много лет назад...,( БЕСПЛАТНО) Забытая родней пенсионерка Валери...,https://rust.litnet.com/uploads/covers/120/169...,https://litnet.com/ru/book/naslednica-divnoi-u...,"Фэнтези, Историческое фэнтези"
4,Вернэлия Хайд,Пленница его величества,‍❤️‍Промо первые 7 дней ‍❤️‍\nБежав от ненавис...,https://rust.litnet.com/uploads/covers/120/174...,https://litnet.com/ru/book/naslednica-divnoi-u...,"Фэнтези, Историческое фэнтези"
...,...,...,...,...,...,...
1775,Алсет Виссон,Сказка для спящего бога,"Иногда всё, что можно сделать - это рассказать...",https://rust.litnet.com/common/img/empty_cover...,https://litnet.com/ru/book/ten-feniksa-b201967,"Фэнтези, Историческое фэнтези"
1776,Василий Чаечашкин,Равноценная вира,"Какую ""виру"" могут заплатить те, кто забирает ...",https://rust.litnet.com/uploads/covers/120/157...,https://litnet.com/ru/book/ten-feniksa-b201967,"Фэнтези, Историческое фэнтези"
1777,Арина Билецкая,Кольцар. Крик воспоминаний,"ЕСТЬ БУКТРЕЙЛЕР\r\n\r\nВоровство наказуемо, но...",https://rust.litnet.com/uploads/covers/120/157...,https://litnet.com/ru/book/ten-feniksa-b201967,"Фэнтези, Историческое фэнтези"
1778,Marie_Lemon,Жзл,Книга про известных людей и их жизнь. Из неско...,https://rust.litnet.com/uploads/covers/120/157...,https://litnet.com/ru/book/ten-feniksa-b201967,"Фэнтези, Историческое фэнтези"


In [10]:
books_df.to_csv(f'{NAME}.csv', index=False)